In [2]:
import pandas as pd
import numpy as np

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# https://baeseongsu.github.io/posts/mimiciii/
# 기초 테이블 확인 : https://mimic.mit.edu/docs/iii/tables/
# 스키마 확인 : https://mit-lcp.github.io/mimic-schema-spy/

# 01 폐렴환자 추출

In [3]:
patient = pd.read_csv('/project/data/PATIENTS.csv')
lab = pd.read_csv('/project/data/LABEVENTS.csv')
diagnoses = pd.read_csv('/project/data/D_ICD_DIAGNOSES.csv') # 질병 정보만 있음 
diagnoses_icd = pd.read_csv('/project/data/DIAGNOSES_ICD.csv') # 환자별 질병 정보 

In [4]:
# 1. 폐와 관련된 질병 추출 (95가지)
diag_pneu = diagnoses[(diagnoses['SHORT_TITLE'].str.contains('pneum')|(diagnoses['SHORT_TITLE'].str.contains('Pneum')))]
diag_pneu.shape

# 2. 폐와 관련된 질병을 가진 환자 추출 -> 14159명
# 가장 많은 폐렴 종류 상위 3개 -> 486, 5070, 48241 (top3_pneu)
pneu_id = diagnoses_icd[diagnoses_icd['ICD9_CODE'].isin(diag_pneu['ICD9_CODE'])].reset_index()
top3_pneu = pneu_id['ICD9_CODE'].value_counts()[:3].index

# 3. ICD9_CODE : 486, 5070, 48241의 정확한 병명 확인
# -> 486 : 
# -> 5070 : 음식 또는 구토물에 의한 폐렴
# -> 48241 : 
diagnoses[diagnoses['ICD9_CODE'].isin(top3_pneu)]

# 4. top3_pneu에 해당하는 환자 추출 
pneu_sub = pneu_id[(pneu_id['ICD9_CODE'].isin(top3_pneu))].drop(['index','ROW_ID','SEQ_NUM'],axis=1).reset_index(drop=True)

# 5. pneu_sub에 한 환자 당 여러 병원기록을 가지고 있을 수 있음
# 예측에 활용할 unique한 환자 수 : 7807
환자id = pneu_sub['SUBJECT_ID'].unique()
len(환자id)

# 6. 추출한 환자id로 환자의 사망, 생존 분포 확인
# -> 사망 : 4777, 생존 : 3030
patient[patient['SUBJECT_ID'].isin(환자id)]['EXPIRE_FLAG'].value_counts()

# 7. 추출한 환자id로 patient에서 환자 정보 추출
환자 = patient[patient['SUBJECT_ID'].isin(환자id)]
# 환자

# 환자.to_csv('폐렴환자.csv', index=False)

(95, 4)

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
5129,5509,48241,Meth sus pneum d/t Staph,Methicillin susceptible pneumonia due to Staph...
5147,5528,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified"
5407,5136,5070,Food/vomit pneumonitis,Pneumonitis due to inhalation of food or vomitus


7807

1    4777
0    3030
Name: EXPIRE_FLAG, dtype: int64

In [5]:
# 8. labevents에서 폐렴환자 추출 : '환자lab'
환자lab = lab[lab['SUBJECT_ID'].isin(환자id)].reset_index(drop=True)

# 9. 'FLAG'열의 결측치를 'nan'으로 설정
환자lab['FLAG'] = 환자lab['FLAG'].fillna('nan')

# 10. 환자의 labevents에서 item별 flag값 확인 
len(환자lab)
환자lab['FLAG'].value_counts()

# 11. 환자lab의 전체 ITEMID 개수 확인
환자lab['ITEMID'].nunique()

# 12. 환자lab -> csv로 저장 
# 환자lab.to_csv('폐렴환자lab.csv', index=False)

# 13. patient의 폐렴 환자가 labevent의 폐렴 환자수보다 8명 더 많음
# 외래환자는 labevent의 'HADM_ID'를 가지지 않음 
외래환자idx = list(set(환자id) - set(환자lab['SUBJECT_ID'].unique()))
외래환자idx

9356349

nan         5931403
abnormal    3406609
delta         18337
Name: FLAG, dtype: int64

690

[60961, 48968, 17674, 9388, 19097, 93114, 58012, 95230]

In [ ]:
admission = pd.read_csv('/project/data/ADMISSIONS.csv')

In [13]:
admission

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,58594,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1
58972,58595,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1
58973,58596,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,RENAL CANCER/SDA,0,1
58974,58597,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,CATHOLIC,MARRIED,WHITE,2128-11-10 23:48:00,2128-11-11 03:16:00,S/P FALL,0,0


# y_(7727,1)

In [14]:
total_data = pd.read_csv('total_data_7727.csv')
sub7727 = total_data['SUBJECT_ID'].unique()

patient = pd.read_csv('폐렴환자.csv')
patient = patient.sort_values(by='SUBJECT_ID')
y_check = patient[patient['SUBJECT_ID'].isin(sub7727)]['EXPIRE_FLAG'].values

In [18]:
## y_(7727,1)이 만들어지는 과정 확인
y_check
y = np.load('y_(7727,1).npy')
y
set(y)-set(y_check)

array([1, 1, 1, ..., 1, 1, 0])

array([1, 1, 1, ..., 1, 1, 0])

set()

In [19]:
y.sum()

4718